In [46]:
import numpy
from csv import reader
from collections import defaultdict
from itertools import chain, combinations

### Konstruksi FP-Tree

In [47]:
class Node:
    def __init__(self, itemName, frequency, parentNode):
        self.itemName = itemName
        self.count = frequency
        self.parent = parentNode
        self.children = {}
        self.next = None

    def increment(self, frequency):
        self.count += frequency

    def display(self, ind=1):
        print('  ' * ind, self.itemName, ' ', self.count)
        for child in list(self.children.values()):
            child.display(ind+1)

In [48]:
def updateHeaderTable(item, targetNode, headerTable):
    if(headerTable[item][1] == None):
        headerTable[item][1] = targetNode
    else:
        currentNode = headerTable[item][1]
        # Traverse to the last node then link it to the target
        while currentNode.next != None:
            currentNode = currentNode.next
        currentNode.next = targetNode

In [49]:
def updateTree(item, parentNode, headerTable):
    if item in parentNode.children:
        # If the item already exists, increment the count
        parentNode.children[item].increment(1)
    else:
        # Create a new branch
        newItemNode = Node(item, 1, parentNode)
        parentNode.children[item] = newItemNode
        # Link the new branch to header table
        updateHeaderTable(item, newItemNode, headerTable)

    return parentNode.children[item]

In [50]:
def constructFPTree(listOfItemset, frequency, minimumSupport):
    headerTable = defaultdict(int)
    # Counting frequency and create header table
    for i, itemset in enumerate(listOfItemset):
        for item in itemset:
            headerTable[item] += frequency[i]

    # Deleting items below minSup
    headerTable = dict((item, supportValue) for item, supportValue in headerTable.items() if supportValue >= minimumSupport)

    if(len(headerTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in headerTable:
        headerTable[item] = [headerTable[item], None]

    # Init Null head node
    initialNode = Node('Null', 1, None)
    # Update FP tree for each cleaned and sorted itemSet
    for _, itemset in enumerate(listOfItemset):
        itemset = [item for item in itemset if item in headerTable]
        itemset.sort(key=lambda item: (-headerTable[item][0], item))

        # Traverse from root to leaf, update tree with given item
        currentNode = initialNode
        for item in itemset:
            currentNode = updateTree(item, currentNode, headerTable)

    return initialNode, headerTable

### Mining FP-Tree

In [51]:
def findPrefixPath(node, prefixPath):
    if node.parent != None:
        prefixPath.append(node.itemName)
        findPrefixPath(node.parent, prefixPath)

In [52]:
def createConditionalPatternBase(item, headerTable):
    # First node in linked list
    nodeOfTree = headerTable[item][1]
    conditionalPaths = []
    frequencyOfEachPath = []

    while nodeOfTree != None:
        prefixPath = []
        # From leaf node all the way to root and reverse the prefix path
        findPrefixPath(nodeOfTree, prefixPath)
        prefixPath = prefixPath[::-1]
        
        if len(prefixPath) > 1:
            # Storing the prefix path and it's corresponding count
            conditionalPaths.append(prefixPath[:len(prefixPath)-1])
            frequencyOfEachPath.append(nodeOfTree.count)

        # Go to next node / item in other branch
        nodeOfTree = nodeOfTree.next

    return conditionalPaths, frequencyOfEachPath

In [53]:
def constructConditionalTree(conditionalPatternBase, frequency, minimumSupport):
    conditionalHeaderTable = defaultdict(int)
    # Counting frequency and create header table
    for i, itemSet in enumerate(conditionalPatternBase):
        for item in itemSet:
            conditionalHeaderTable[item] += frequency[i]

    # Deleting items below minSup
    conditionalHeaderTable = dict((item, supportValue) for item, supportValue in conditionalHeaderTable.items() if supportValue >= minimumSupport)

    if(len(conditionalHeaderTable) == 0):
        return None, None

    # HeaderTable column [Item: [frequency, headNode]]
    for item in conditionalHeaderTable:
        conditionalHeaderTable[item] = [conditionalHeaderTable[item], None]

    # Init Null head node
    conditionalInitialNode = Node('Null', 1, None)

    conditionalPatternBaseExtracted = []
    for itemset, freq in zip(conditionalPatternBase, frequency):
        conditionalPatternBaseExtracted.extend([itemset.copy() for _ in range(freq)])

    # Update FP tree for each cleaned and sorted itemSet
    for _, itemset in enumerate(conditionalPatternBaseExtracted):
        itemset = [item for item in itemset if item in conditionalHeaderTable]
        # sorting by ascending or alphabet

        # Traverse from root to leaf, update tree with given item
        currentNode = conditionalInitialNode
        for item in itemset:
            currentNode = updateTree(item, currentNode, conditionalHeaderTable)

    return conditionalInitialNode, conditionalHeaderTable

In [54]:
def miningTrees(headerTable, minimumSupport, prefix, freqItemsetList):
    print("=====start=====")
    # Sort the items with frequency and create a list
    # sortedItemList = [item[0] for item in sorted(list(headerTable.items()), key=lambda p:p[1][0])]
    itemlistSorted = [item[0] for item in sorted(headerTable.items(), key=lambda item: (-item[1][0], item[0]))]
    print("itemlistSorted", itemlistSorted)
    reversedItemList = itemlistSorted[::-1]

    # Start with the lowest frequency
    for item in itemlistSorted:  
        # Pattern growth is achieved by the concatenation of suffix pattern with frequent patterns generated from conditional FP-tree
        newFreqItemset = prefix.copy()
        newFreqItemset.add(item)
        print("Mining item", item)
        print("NewFreqSet", newFreqItemset)
        freqItemsetList.append(newFreqItemset)
        print("FreqPatternItemset", freqItemsetList)

        # Find all prefix path, constrcut conditional pattern base
        conditionalPatternBase, frequency = createConditionalPatternBase(item, headerTable)
        print("conditionalPattBase of item", item, "is", conditionalPatternBase)
        # Construct conditonal FP Tree with conditional pattern base
        conditionalTree, newHeaderTable = constructConditionalTree(conditionalPatternBase, frequency, minimumSupport)
        
        if newHeaderTable != None:
            print("trigger")
            print(conditionalTree.display())
            # Mining recursively on the tree
            miningTrees(newHeaderTable, minimumSupport, newFreqItemset, freqItemsetList)

## Association Rule

In [55]:
def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

In [56]:
def getSupport(testSet, itemSetList):
    count = 0
    for itemSet in itemSetList:
        if(set(testSet).issubset(itemSet)):
            count += 1
    return count

In [57]:
def associationRule(freqItemSet, itemSetList, minConf):
    rules = []

    for itemSet in freqItemSet:
        subsets = powerset(itemSet)
        itemSetSup = getSupport(itemSet, itemSetList)
        
        for s in subsets:
            confidence = float(itemSetSup / getSupport(s, itemSetList))
            if(confidence > minConf):
                rules.append([set(s), set(itemSet.difference(s)), confidence])
    
    return rules

### Trigger the Algorithm

In [58]:
def getDatasetFromFile(fname):
    listOfItemset = []
    frequencyOfTransaction = []
    
    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            listOfItemset.append(line)
            frequencyOfTransaction.append(1)

    return listOfItemset, frequencyOfTransaction

In [59]:
def fpgrowthFromFile(filename, minimumSupportRatio, minimumConfidence):
    listOfItemset, frequency = getDatasetFromFile(filename)
    minimumSupport = len(listOfItemset) * minimumSupportRatio

    fpTree, headerTable = constructFPTree(listOfItemset, frequency, minimumSupport)
    # print(fpTree.display())
    if(fpTree.children == None):
        print('No frequent item set')
    else:
        freqentItemset = []
        miningTrees(headerTable, minimumSupport, set(), freqentItemset)
        rules = associationRule(freqentItemset, listOfItemset, minimumConfidence)
        return freqentItemset, rules

In [60]:
freqItems, rules = fpgrowthFromFile("data/example.csv", 0.6, 0.1)
print("\n")
# print("Frequent pattern itemset: ")
# freqItems

print("Rules: ")
rules

=====start=====
itemlistSorted ['c', 'f', 'a', 'b', 'm', 'p']
Mining item c
NewFreqSet {'c'}
FreqPatternItemset [{'c'}]
conditionalPattBase of item c is []
Mining item f
NewFreqSet {'f'}
FreqPatternItemset [{'c'}, {'f'}]
conditionalPattBase of item f is [['c']]
trigger
   Null   1
     c   3
None
=====start=====
itemlistSorted ['c']
Mining item c
NewFreqSet {'f', 'c'}
FreqPatternItemset [{'c'}, {'f'}, {'f', 'c'}]
conditionalPattBase of item c is []
Mining item a
NewFreqSet {'a'}
FreqPatternItemset [{'c'}, {'f'}, {'f', 'c'}, {'a'}]
conditionalPattBase of item a is [['c', 'f']]
trigger
   Null   1
     c   3
       f   3
None
=====start=====
itemlistSorted ['c', 'f']
Mining item c
NewFreqSet {'a', 'c'}
FreqPatternItemset [{'c'}, {'f'}, {'f', 'c'}, {'a'}, {'a', 'c'}]
conditionalPattBase of item c is []
Mining item f
NewFreqSet {'f', 'a'}
FreqPatternItemset [{'c'}, {'f'}, {'f', 'c'}, {'a'}, {'a', 'c'}, {'f', 'a'}]
conditionalPattBase of item f is [['c']]
trigger
   Null   1
     c   3
None

[[{'f'}, {'c'}, 0.75],
 [{'c'}, {'f'}, 0.75],
 [{'a'}, {'c'}, 1.0],
 [{'c'}, {'a'}, 0.75],
 [{'f'}, {'a'}, 0.75],
 [{'a'}, {'f'}, 1.0],
 [{'f'}, {'a', 'c'}, 0.75],
 [{'a'}, {'c', 'f'}, 1.0],
 [{'c'}, {'a', 'f'}, 0.75],
 [{'a', 'f'}, {'c'}, 1.0],
 [{'c', 'f'}, {'a'}, 1.0],
 [{'a', 'c'}, {'f'}, 1.0],
 [{'m'}, {'a'}, 1.0],
 [{'a'}, {'m'}, 1.0],
 [{'m'}, {'a', 'c'}, 1.0],
 [{'a'}, {'c', 'm'}, 1.0],
 [{'c'}, {'a', 'm'}, 0.75],
 [{'a', 'm'}, {'c'}, 1.0],
 [{'c', 'm'}, {'a'}, 1.0],
 [{'a', 'c'}, {'m'}, 1.0],
 [{'m'}, {'a', 'f'}, 1.0],
 [{'f'}, {'a', 'm'}, 0.75],
 [{'a'}, {'f', 'm'}, 1.0],
 [{'f', 'm'}, {'a'}, 1.0],
 [{'a', 'm'}, {'f'}, 1.0],
 [{'a', 'f'}, {'m'}, 1.0],
 [{'m'}, {'a', 'c', 'f'}, 1.0],
 [{'f'}, {'a', 'c', 'm'}, 0.75],
 [{'a'}, {'c', 'f', 'm'}, 1.0],
 [{'c'}, {'a', 'f', 'm'}, 0.75],
 [{'f', 'm'}, {'a', 'c'}, 1.0],
 [{'a', 'm'}, {'c', 'f'}, 1.0],
 [{'c', 'm'}, {'a', 'f'}, 1.0],
 [{'a', 'f'}, {'c', 'm'}, 1.0],
 [{'c', 'f'}, {'a', 'm'}, 1.0],
 [{'a', 'c'}, {'f', 'm'}, 1.0],
 [{'a', 